<a href="https://colab.research.google.com/github/Gebreyowhans/Deep_Learning/blob/main/Subclassing_In_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf


In [4]:
from tensorflow import keras
from tensorflow.keras import layers

**A simple subclassed model**

# When we subclass a model:

1. In the** __init__()** method, define the layers the model will use.
2. In the **call()** method, define the forward pass of the model, reusing the layers previously created.
3. Instantiate your subclass, and call it on data to create its weights.

In [7]:
class CustomerTicketModel(keras.Model):

  def __init__(self,num_departments):
    super().__init__() #calling the super() constructor!

    # Defining sublayers in the constructor.
    self.concate_layer=layers.Concatenate()
    self.mixing_layer=layers.Dense(64, activation="relu")
    self.priority_scorer = layers.Dense(1, activation="sigmoid")
    self.department_classifier = layers.Dense(
        num_departments, activation="softmax")
    
  #Define the forward pass in the call() method.
  def call(self,inputs):
    title=inputs["title"]
    text_body=inputs["text_body"]
    tags = inputs["tags"]
    features=self.concate_layer([title,text_body,tags])
    features=self.mixing_layer(features)
    priority=self.priority_scorer(features)
    department=self.department_classifier(features)

    return priority,department
    

# Note 
Once we’ve defined the model, we can instantiate it. Note
that it will only create its weights the first time we call it on
some data, much like Layer subclasses:

In [8]:
model = CustomerTicketModel(num_departments=4)

In [9]:
# since we didn't give the input to the model ,the weight will not be created
model.weights

[]

*Creating dummy input and output data as we did in the functional api section*





In [12]:

import numpy as np
num_samples = 1280
vocabulary_size = 10000
num_tags = 100
num_departments = 4

# Dummy input data
title_data=np.random.randint(0,2,size=(num_samples,vocabulary_size))
text_body_data=np.random.randint(0,2,size=(num_samples,vocabulary_size))
tags_data=np.random.randint(0,2,size=(num_samples,num_tags))

# Dummy target data
priority_data=np.random.random(size=(num_samples,1))
department_data=np.random.randint(0,2,size=(num_samples,num_departments))


In [13]:
priority, department = model(
{"title": title_data, "text_body": text_body_data, "tags": tags_data})

In [14]:
priority

<tf.Tensor: shape=(1280, 1), dtype=float32, numpy=
array([[0.43043265],
       [0.774653  ],
       [0.49542344],
       ...,
       [0.640006  ],
       [0.7406447 ],
       [0.43854406]], dtype=float32)>

*Compile the sub class model created before*

In [15]:
#The structure of what you pass as the loss and metrics arguments must match exactly what gets returned by call()—
model.compile(optimizer="rmsprop",
              loss=["mean_squared_error", "categorical_crossentropy"],
              metrics=[["mean_absolute_error"], ["accuracy"]])

In [16]:
# fit the model

#The structure of the input data must match exactly what is expected by the call() method —here, a dict with keys title, text_body, and tags.
model.fit({"title": title_data,
           "text_body": text_body_data,
           "tags": tags_data},[priority_data, department_data],#The structure of the target data must match exactly what is returned by the call() method —here, a list of two elements.
          epochs=1)

40/40 [==============================] - 2s 23ms/step - loss: 34.6805 - output_1_loss: 0.3211 - output_2_loss: 34.3594 - output_1_mean_absolute_error: 0.4860 - output_2_accuracy: 0.1914


In [17]:
#evaluate the subclass model
model.evaluate({"title": title_data,
                "text_body": text_body_data,
                "tags": tags_data},
               [priority_data, department_data])

40/40 [==============================] - 1s 15ms/step - loss: 37.7040 - output_1_loss: 0.3293 - output_2_loss: 37.3747 - output_1_mean_absolute_error: 0.4937 - output_2_accuracy: 0.1281


[37.70402908325195,
 0.32930806279182434,
 37.37471008300781,
 0.49371036887168884,
 0.12812499701976776]

In [18]:
# Generate predictions
priority_preds, department_preds = model.predict({"title": title_data,
                                                  "text_body": text_body_data,
                                                  "tags": tags_data})

40/40 [==============================] - 1s 10ms/step


# Note
A subclassed model is a piece of bytecode—a
Python class with a call() method that contains raw code, so it will not plot the model in graphical form  or access layers connectivity information unlike the functional api approach

In [19]:
keras.utils.plot_model(model,"Subclassed Model",show_shapes=True)

ValueError: ignored